In [2]:
from pyspark.sql import SparkSession

spark = SparkSession. \
    builder. \
    config("spark.sql.warehouse.dir", "C:/data/warehouse"). \
    config("spark.sql.catalogImplementation", "hive"). \
    enableHiveSupport(). \
    getOrCreate()


In [3]:
orders_df = spark.read.csv("C:/data/orders_sh.csv", header=True, inferSchema=True)

In [4]:
orders_df.createOrReplaceTempView("orders_tmp")

In [5]:
spark.sql("select * from orders_tmp").show(5)

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|   00:00.0|      11599|         CLOSED|
|       2|   00:00.0|        256|PENDING_PAYMENT|
|       3|   00:00.0|      12111|       COMPLETE|
|       4|   00:00.0|       8827|         CLOSED|
|       5|   00:00.0|      11318|       COMPLETE|
+--------+----------+-----------+---------------+
only showing top 5 rows



In [6]:
spark.sql("create database if not exists spark_sql_db")

DataFrame[]

In [7]:
spark.sql("show databases").show()

+------------+
|   namespace|
+------------+
|     default|
|spark_sql_db|
+------------+



In [8]:
spark.sql("create table if not exists spark_sql_db.orders_managed as select * from orders_tmp")

DataFrame[]

In [9]:
spark.sql("show tables in spark_sql_db").show()

+------------+--------------+-----------+
|   namespace|     tableName|isTemporary|
+------------+--------------+-----------+
|spark_sql_db|orders_managed|      false|
|            |    orders_tmp|       true|
+------------+--------------+-----------+



In [10]:
spark.sql("select * from spark_sql_db.orders_managed").show(5)

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|   00:00.0|      11599|         CLOSED|
|       2|   00:00.0|        256|PENDING_PAYMENT|
|       3|   00:00.0|      12111|       COMPLETE|
|       4|   00:00.0|       8827|         CLOSED|
|       5|   00:00.0|      11318|       COMPLETE|
+--------+----------+-----------+---------------+
only showing top 5 rows



In [11]:
spark.sql("describe extended spark_sql_db.orders_managed").show(30, False)

+----------------------------+----------------------------------------------------------+-------+
|col_name                    |data_type                                                 |comment|
+----------------------------+----------------------------------------------------------+-------+
|order_id                    |int                                                       |NULL   |
|order_date                  |string                                                    |NULL   |
|customer_id                 |int                                                       |NULL   |
|order_status                |string                                                    |NULL   |
|                            |                                                          |       |
|# Detailed Table Information|                                                          |       |
|Catalog                     |spark_catalog                                             |       |
|Database           

In [14]:
spark.sql("create external table if not exists spark_sql_db.orders_external (order_id int, order_date string, order_customer_id int, order_status string) row format delimited fields terminated by ',' location 'file:///C:/data/warehouse/spark_sql_db.db/orders_managed'")

DataFrame[]

In [18]:
spark.sql("describe extended spark_sql_db.orders_external").show(30, False)

+----------------------------+----------------------------------------------------------+-------+
|col_name                    |data_type                                                 |comment|
+----------------------------+----------------------------------------------------------+-------+
|order_id                    |int                                                       |NULL   |
|order_date                  |string                                                    |NULL   |
|order_customer_id           |int                                                       |NULL   |
|order_status                |string                                                    |NULL   |
|                            |                                                          |       |
|# Detailed Table Information|                                                          |       |
|Catalog                     |spark_catalog                                             |       |
|Database           

In [15]:
spark.sql("""create table spark_sql_db.orders_ext2
          (order_id int, order_date string, order_customer_id int, order_status string)
          using csv options (path 'C:/data/orders_sh.csv')""")

DataFrame[]

In [17]:
spark.sql("show tables in spark_sql_db").show()

+------------+---------------+-----------+
|   namespace|      tableName|isTemporary|
+------------+---------------+-----------+
|spark_sql_db|    orders_ext2|      false|
|spark_sql_db|orders_external|      false|
|spark_sql_db| orders_managed|      false|
|            |     orders_tmp|       true|
+------------+---------------+-----------+



In [19]:
spark.sql("describe extended spark_sql_db.orders_ext2").show(30, False)

+----------------------------+---------------------------------------------------------+-------+
|col_name                    |data_type                                                |comment|
+----------------------------+---------------------------------------------------------+-------+
|order_id                    |int                                                      |NULL   |
|order_date                  |string                                                   |NULL   |
|order_customer_id           |int                                                      |NULL   |
|order_status                |string                                                   |NULL   |
|                            |                                                         |       |
|# Detailed Table Information|                                                         |       |
|Catalog                     |spark_catalog                                            |       |
|Database                    |

In [20]:
spark.sql("drop table if exists spark_sql_db.orders_managed")

DataFrame[]

In [22]:
spark.sql("select * from spark_sql_db.orders_ext2").show(5)

+--------+----------+-----------------+---------------+
|order_id|order_date|order_customer_id|   order_status|
+--------+----------+-----------------+---------------+
|    NULL|order_date|             NULL|   order_status|
|       1|   00:00.0|            11599|         CLOSED|
|       2|   00:00.0|              256|PENDING_PAYMENT|
|       3|   00:00.0|            12111|       COMPLETE|
|       4|   00:00.0|             8827|         CLOSED|
+--------+----------+-----------------+---------------+
only showing top 5 rows



In [ ]:
spark.sql("drop table if exists spark_sql_db.orders_ext2")